In [ ]:
!pip list
!pip install --upgrade google-api-python-client
!pip install --upgrade google-api-core
!pip install --upgrade google-cloud
!pip install google-cloud-vision==2.7.1
!pip install IPython

from os import path
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import rescale, resize
from pyxelate.pyx import Pyx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.cloud import vision
import os

In [40]:
def plot(subplots=[], save_as=None, fig_h=9):
    """Plotting helper function"""
    fig, ax = plt.subplots(int(np.ceil(len(subplots) / 3)), 
                           min(3, len(subplots)), 
                           figsize=(18, fig_h))
    if len(subplots) == 1:
        ax = [ax]
    else:
        ax = ax.ravel()
    for i, subplot in enumerate(subplots):
        if isinstance(subplot, dict):
            ax[i].set_title(subplot["title"])
            ax[i].imshow(subplot["image"])
        else:
            ax[i].imshow(subplot)
    fig.tight_layout()
    plt.show()

In [41]:
def pixel_image(path):
    # detect image using Vision API
    if detect_image(path):
      image = io.imread(path)  

      downsample_by = 14 
      palette = 7

      # 1) Instantiate Pyx transformer
      pyx = Pyx(factor=downsample_by, palette=palette)

      # 2) fit an image, allow Pyxelate to learn the color
      pyx.fit(image)

      # 3) transform image to pixel art using the learned color
      new_image = pyx.transform(image)

      plot([image, new_image], "new_image.jpeg")

      io.imsave("new_image.jpeg", new_image)

    else :
      print('No racy or adult violence!')

In [42]:
def detect_image(path):
    # Google SafeSearch Detection API detects explicit content such as adult content or violent content within an image. 
    # This feature uses five categories (adult, spoof, medical, violence, and racy) and returns the likelihood that each is present in a given image
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "My Project-0e5eb54f3bdb.json"
    import io as ios
    client = vision.ImageAnnotatorClient()

    with ios.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.safe_search_detection(image=image)
    safe = response.safe_search_annotation

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    # print('Safe search:')

    # print('adult: {}'.format(likelihood_name[safe.adult]))
    # print('medical: {}'.format(likelihood_name[safe.medical]))
    # print('spoofed: {}'.format(likelihood_name[safe.spoof]))
    # print('violence: {}'.format(likelihood_name[safe.violence]))
    # print('racy: {}'.format(likelihood_name[safe.racy]))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return safe.adult >=3 or safe.medical >=3 or safe.spoof >=3 or safe.violence >=3 or safe.racy >=3


In [51]:
if __name__ == "__main__":
    pixel_image("test2.jpeg")

No racy or adult violence!
